In [ ]:
import numpy as np
import pandas as pd
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense, GRU, Embedding
from keras.callbacks import EarlyStopping, ModelCheckpoint
import re


In [ ]:
def text_cleaner(text):
    # lower case text
    newString = text.lower()
    newString = re.sub(r"'s\b","",newString)
    # remove punctuations
    newString = re.sub("[^a-zA-Z]", " ", newString)
    long_words=[]
    # remove short word
    for i in newString.split():
        if len(i)>=3:
            long_words.append(i)
    return (" ".join(long_words)).strip()

# Open the file and read its contents
with open('/content/lyrics_dataset.txt', 'r') as file:
    data_text = file.read()

# preprocess the text
data_new = text_cleaner(data_text)

In [ ]:
def create_seq(text):
    length = 30
    sequences = list()
    for i in range(length, len(text)):
        # select sequence of tokens
        seq = text[i-length:i+1]
        # store
        sequences.append(seq)
    print('Total Sequences: %d' % len(sequences))
    return sequences

# create sequences
sequences = create_seq(data_new)

Total Sequences: 311172


In [ ]:
# create a character mapping index
chars = sorted(list(set(data_new)))
mapping = dict((c, i) for i, c in enumerate(chars))

def encode_seq(seq):
    sequences = list()
    for line in seq:
        # integer encode line
        encoded_seq = [mapping[char] for char in line]
        # store
        sequences.append(encoded_seq)
    return sequences

# encode the sequences
sequences = encode_seq(sequences)

In [ ]:
from sklearn.model_selection import train_test_split

# vocabulary size
vocab = len(mapping)
sequences = np.array(sequences)
# create X and y
X, y = sequences[:,:-1], sequences[:,-1]
# one hot encode y
y = to_categorical(y, num_classes=vocab)
# create train and validation sets
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

print('Train shape:', X_tr.shape, 'Val shape:', X_val.shape)

Train shape: (280054, 30) Val shape: (31118, 30)


In [ ]:
# define model
model = Sequential()
model.add(Embedding(vocab, 50, input_length=30, trainable=True))
model.add(GRU(150, recurrent_dropout=0.1, dropout=0.1))
model.add(Dense(vocab, activation='softmax'))
print(model.summary())


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 30, 50)            1350      
                                                                 
 gru_1 (GRU)                 (None, 150)               90900     
                                                                 
 dense_1 (Dense)             (None, 27)                4077      
                                                                 
Total params: 96327 (376.28 KB)
Trainable params: 96327 (376.28 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
# compile the model
model.compile(loss='categorical_crossentropy', metrics=['acc'], optimizer='adam')


In [ ]:
# fit the model
model.fit(X_tr, y_tr, epochs=10, verbose=2, validation_data=(X_val, y_val))

Epoch 1/10
8752/8752 - 735s - loss: 1.5214 - acc: 0.5316 - val_loss: 1.4266 - val_acc: 0.5557 - 735s/epoch - 84ms/step
Epoch 2/10
8752/8752 - 724s - loss: 1.4292 - acc: 0.5561 - val_loss: 1.3840 - val_acc: 0.5695 - 724s/epoch - 83ms/step
Epoch 3/10
8752/8752 - 732s - loss: 1.3908 - acc: 0.5649 - val_loss: 1.3503 - val_acc: 0.5810 - 732s/epoch - 84ms/step
Epoch 4/10
8752/8752 - 724s - loss: 1.3685 - acc: 0.5708 - val_loss: 1.3355 - val_acc: 0.5849 - 724s/epoch - 83ms/step
Epoch 5/10
8752/8752 - 725s - loss: 1.3554 - acc: 0.5753 - val_loss: 1.3247 - val_acc: 0.5916 - 725s/epoch - 83ms/step
Epoch 6/10
8752/8752 - 725s - loss: 1.3458 - acc: 0.5770 - val_loss: 1.3161 - val_acc: 0.5927 - 725s/epoch - 83ms/step
Epoch 7/10
8752/8752 - 729s - loss: 1.3380 - acc: 0.5793 - val_loss: 1.3063 - val_acc: 0.5938 - 729s/epoch - 83ms/step
Epoch 8/10
8752/8752 - 751s - loss: 1.3335 - acc: 0.5809 - val_loss: 1.3060 - val_acc: 0.5940 - 751s/epoch - 86ms/step
Epoch 9/10
8752/8752 - 723s - loss: 1.3297 - acc

In [ ]:
import numpy as np

def generate_seq(model, mapping, seq_length, seed_text, n_chars):
    in_text = seed_text
    # generate a fixed number of characters
    for _ in range(n_chars):
        # encode the characters as integers
        encoded = [mapping[char] for char in in_text]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # predict character
        yhat = model.predict(encoded, verbose=0)
        # map predicted integer to character
        out_char = ''
        for char, index in mapping.items():
            if index == np.argmax(yhat):
                out_char = char
                break
        # append to input
        in_text += out_char
    return in_text


In [ ]:
inp = " "
print(len(inp))
print(generate_seq(model,mapping,30,inp.lower(),15))

1
 river and the s


In [ ]:
from math import exp
from keras.preprocessing.sequence import pad_sequences

# Calculate Perplexity
def calculate_perplexity(model, X_val, y_val):
    # Evaluate the model on validation set
    loss = model.evaluate(X_val, y_val, verbose=0)
    cross_entropy = loss[0]
    # Calculate perplexity
    perplexity = exp(cross_entropy)
    return perplexity

# Generate Text Samples
def generate_text_samples(model, mapping, seq_length, seed_texts, n_chars):
    generated_texts = []
    for seed_text in seed_texts:
        generated_text = generate_seq(model, mapping, seq_length, seed_text.lower(), n_chars)
        generated_texts.append(generated_text)
    return generated_texts

# Calculate perplexity
perplexity = calculate_perplexity(model, X_val, y_val)
print("Perplexity:", perplexity)

# Generate text samples
seed_texts = ["hello", "the cat", "machine learning"]
generated_texts = generate_text_samples(model, mapping, 30, seed_texts, 100)
for i, text in enumerate(generated_texts):
    print(f"Generated Text {i+1}: {text}")


Perplexity: 3.6661532688250107
Generated Text 1: hellow the same say what you need say when you and the same say what you need say when you and the same s
Generated Text 2: the catch the same say what you need say when you and the same say what you need say when you and the same 
Generated Text 3: machine learning the same say what you need say when you and the same say what you need say when you and the same sa


In [ ]:
model.save('text quality.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
